In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import string
import random

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from nltk.collocations import *
#from nltk import FreqDist, word_tokenize

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing

from keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Activation, LSTM, GRU, Dense, GlobalMaxPool1D, Embedding, Dropout, Conv1D
from keras.models import Sequential, Model
from keras import backend

Using TensorFlow backend.


In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
df = pd.read_csv('data/drug_review_clean.csv')

In [4]:
drugs = df

In [29]:
top_10_conditions[top_10_conditions.index=='Birth Control'].drugName.value_counts()[:1].keys()[0]

'Etonogestrel'

In [115]:
top_10_conditions = drugs.set_index('condition').loc[drugs.condition.value_counts()[:10].keys()]

In [116]:
#top_10_conditions.groupby(top_10_conditions.index)['drugName'].value_counts()

In [117]:
top_drug_for_condition = []
for i in top_10_conditions.index.unique():
    drug_name = top_10_conditions[top_10_conditions.index==i].drugName.value_counts()[:1].keys()[0]
    top_drug_for_condition.append(drug_name)

In [118]:
top_drug_for_condition

['Etonogestrel',
 'Bupropion',
 'Tramadol',
 'Escitalopram',
 'Isotretinoin',
 'Lamotrigine',
 'Zolpidem',
 'Phentermine',
 'Bupropion / naltrexone',
 'Lisdexamfetamine']

In [119]:
top_drugs = pd.DataFrame()
for i in top_drug_for_condition:
    top_drugs = pd.concat([top_drugs, top_10_conditions[top_10_conditions.drugName==i]])

In [129]:
def hot_tokenization(text, num_words=3000):
    tokenizer = Tokenizer(num_words)
    tokenizer.fit_on_texts(text)
    return tokenizer.texts_to_matrix(text, mode='binary')

def hot_pad_sequences(text, num_words=3000, maxlen=100):    
    tokenizer = Tokenizer(num_words)
    tokenizer.fit_on_texts(list(text))
    list_tokenized_headlines = tokenizer.texts_to_sequences(text)
    return sequence.pad_sequences(list_tokenized_headlines, maxlen) 

def hot_label_encoding(labels):
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    labels_cat = le.transform(labels)
    return to_categorical(labels_cat) 

In [121]:
X_t = hot_tokenization(top_drugs.lemm_review, 3000)
y = hot_label_encoding(top_drugs.drugName)
train, test, label_train, label_test = train_test_split(X_t, y, test_size=0.2, random_state=42)

In [ ]:
X_t = hot_pad_sequences(top_drugs.lemm_review, 3000, maxlen=100)
y = hot_label_encoding(top_drugs.drugName)
train, test, label_train, label_test = train_test_split(X_t, y, test_size=0.2, random_state=42)

In [124]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(3000,)))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=optimizers.SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [127]:
history = model.fit(train, label_train, epochs=20, batch_size=256, validation_split=0.1)
history_dict = history.history

Train on 9332 samples, validate on 1037 samples
Epoch 1/20
9332/9332 [==============================] - 1s 143us/step - loss: 0.6209 - acc: 0.8542 - val_loss: 0.6526 - val_acc: 0.8351
Epoch 2/20
9332/9332 [==============================] - 1s 152us/step - loss: 0.6070 - acc: 0.8586 - val_loss: 0.6394 - val_acc: 0.8447
Epoch 3/20
9332/9332 [==============================] - 1s 153us/step - loss: 0.5936 - acc: 0.8623 - val_loss: 0.6267 - val_acc: 0.8476
Epoch 4/20
9332/9332 [==============================] - 1s 142us/step - loss: 0.5806 - acc: 0.8671 - val_loss: 0.6143 - val_acc: 0.8534
Epoch 5/20
9332/9332 [==============================] - 1s 153us/step - loss: 0.5680 - acc: 0.8693 - val_loss: 0.6021 - val_acc: 0.8554
Epoch 6/20
9332/9332 [==============================] - 1s 153us/step - loss: 0.5558 - acc: 0.8724 - val_loss: 0.5903 - val_acc: 0.8621
Epoch 7/20
9332/9332 [==============================] - 1s 147us/step - loss: 0.5440 - acc: 0.8755 - val_loss: 0.5795 - val_acc: 0.8660


In [130]:
X_t = hot_pad_sequences(top_drugs.lemm_review, 3000, maxlen=100)
y = hot_label_encoding(top_drugs.drugName)
train, test, label_train, label_test = train_test_split(X_t, y, test_size=0.2, random_state=42)

In [144]:
backend.clear_session()

model = Sequential()
model.add(Embedding(3000, 80, input_length=100))
model.add(Conv1D(60, 2, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 80)           240000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 99, 60)            9660      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 60)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1830      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
Total para

In [145]:
history = model.fit(train, label_train, epochs=6, validation_split=0.1, batch_size=80)

Train on 9332 samples, validate on 1037 samples
Epoch 1/6
9332/9332 [==============================] - 26s 3ms/step - loss: 1.6500 - acc: 0.4273 - val_loss: 1.0436 - val_acc: 0.6731
Epoch 2/6
9332/9332 [==============================] - 24s 3ms/step - loss: 0.6759 - acc: 0.7991 - val_loss: 0.4071 - val_acc: 0.8737
Epoch 3/6
9332/9332 [==============================] - 26s 3ms/step - loss: 0.3680 - acc: 0.8871 - val_loss: 0.3079 - val_acc: 0.8949
Epoch 4/6
9332/9332 [==============================] - 30s 3ms/step - loss: 0.2746 - acc: 0.9189 - val_loss: 0.2876 - val_acc: 0.9026
Epoch 5/6
9332/9332 [==============================] - 32s 3ms/step - loss: 0.2143 - acc: 0.9359 - val_loss: 0.2865 - val_acc: 0.9055
Epoch 6/6
9332/9332 [==============================] - 33s 4ms/step - loss: 0.1696 - acc: 0.9495 - val_loss: 0.2866 - val_acc: 0.9065


In [147]:
drugs[drugs.condition=='Birth Control'].drugName.value_counts()[:10]

Etonogestrel                          4394
Ethinyl estradiol / norethindrone     3081
Levonorgestrel                        2884
Nexplanon                             2883
Ethinyl estradiol / levonorgestrel    2107
Ethinyl estradiol / norgestimate      2097
Implanon                              1496
Mirena                                1320
Skyla                                 1074
Lo Loestrin Fe                         896
Name: drugName, dtype: int64

In [153]:
top_birthcontrol_drugs = drugs.set_index('drugName').loc[drugs[drugs.condition=='Birth Control'].drugName.value_counts()[:10].keys()]

In [164]:
X_t = hot_tokenization(top_birthcontrol_drugs.lemm_review, 3000)
y = hot_label_encoding(top_birthcontrol_drugs.index)
train, test, label_train, label_test = train_test_split(X_t, y, test_size=0.2, random_state=42)

In [165]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(3000,)))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=optimizers.SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [166]:
history = model.fit(train, label_train, epochs=40, batch_size=256, validation_split=0.1)
history_dict = history.history

Train on 18763 samples, validate on 2085 samples
Epoch 1/40
18763/18763 [==============================] - 5s 284us/step - loss: 2.2966 - acc: 0.1367 - val_loss: 2.2754 - val_acc: 0.1923
Epoch 2/40
18763/18763 [==============================] - 3s 156us/step - loss: 2.2524 - acc: 0.2220 - val_loss: 2.2380 - val_acc: 0.2331
Epoch 3/40
18763/18763 [==============================] - 3s 145us/step - loss: 2.2134 - acc: 0.2533 - val_loss: 2.2023 - val_acc: 0.2513
Epoch 4/40
18763/18763 [==============================] - 3s 143us/step - loss: 2.1740 - acc: 0.2727 - val_loss: 2.1646 - val_acc: 0.2667
Epoch 5/40
18763/18763 [==============================] - 3s 149us/step - loss: 2.1316 - acc: 0.2897 - val_loss: 2.1227 - val_acc: 0.2758
Epoch 6/40
18763/18763 [==============================] - 3s 145us/step - loss: 2.0850 - acc: 0.2968 - val_loss: 2.0769 - val_acc: 0.2873
Epoch 7/40
18763/18763 [==============================] - 3s 150us/step - loss: 2.0354 - acc: 0.3074 - val_loss: 2.0276 - v

In [169]:
X_t = hot_pad_sequences(top_birthcontrol_drugs.lemm_review, 3000, maxlen=100)
y = hot_label_encoding(top_birthcontrol_drugs.index)
train, test, label_train, label_test = train_test_split(X_t, y, test_size=0.2, random_state=42)

In [170]:
backend.clear_session()

model = Sequential()
model.add(Embedding(3000, 80, input_length=100))
model.add(Conv1D(60, 2, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.4))
model.add(Dense(30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 80)           240000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 99, 60)            9660      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 60)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1830      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                310       
Total para

In [171]:
history = model.fit(train, label_train, epochs=6, validation_split=0.1, batch_size=80)

Train on 18763 samples, validate on 2085 samples
Epoch 1/6
18763/18763 [==============================] - 42s 2ms/step - loss: 1.8294 - acc: 0.3283 - val_loss: 1.2565 - val_acc: 0.4830
Epoch 2/6
18763/18763 [==============================] - 55s 3ms/step - loss: 1.2582 - acc: 0.4799 - val_loss: 1.0526 - val_acc: 0.5314
Epoch 3/6
18763/18763 [==============================] - 43s 2ms/step - loss: 1.0885 - acc: 0.5352 - val_loss: 0.9845 - val_acc: 0.5444
Epoch 4/6
18763/18763 [==============================] - 40s 2ms/step - loss: 1.0138 - acc: 0.5567 - val_loss: 0.9642 - val_acc: 0.5525
Epoch 5/6
18763/18763 [==============================] - 54s 3ms/step - loss: 0.9688 - acc: 0.5661 - val_loss: 0.9561 - val_acc: 0.5511
Epoch 6/6
18763/18763 [==============================] - 89s 5ms/step - loss: 0.9322 - acc: 0.5751 - val_loss: 0.9448 - val_acc: 0.5549


In [192]:
def hot_pad_sequences(text, num_words=2, maxlen=10):    
    tokenizer = Tokenizer(num_words)
    tokenizer.fit_on_texts(list(text))
    list_tokenized_headlines = tokenizer.texts_to_sequences(text)
    return sequence.pad_sequences(list_tokenized_headlines, maxlen)

In [193]:
hot_pad_sequences(['sean','bill'])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)